In [1]:
import torch
from torchvision import datasets, transforms

In [2]:
!pip install torch torchvision


In [2]:
!pip install spikingjelly


In [4]:
import torch
from torchvision import datasets, transforms

# Load train and validation datasets
train_dataset = torch.load("final_project/train_dataset.pth")
val_dataset = torch.load("final_project/val_dataset.pth")

# Explore the datasets
print(len(train_dataset))
print(len(val_dataset))


7000
3000


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import spikingjelly.clock_driven.encoding as encoding

class LIFNeuron(nn.Module):
    def __init__(self, tau_syn=5.0, v_threshold=1.0, v_reset=0.0, initial_v=-1.0):
        super(LIFNeuron, self).__init__()
        self.tau_syn = tau_syn
        self.v_threshold = v_threshold
        self.v_reset = v_reset
        self.initial_v = initial_v
        self.synaptic_current = torch.zeros(1)

    def forward(self, synaptic_input):
        self.synaptic_current = torch.exp(-1.0 / self.tau_syn) * self.synaptic_current + synaptic_input
        spike = (self.synaptic_current >= self.v_threshold).float()
        self.synaptic_current = torch.where(spike > 0, torch.tensor(self.v_reset), self.synaptic_current)
        return spike

In [6]:
class SNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SNN, self).__init__()

        # Define LIF neurons and encoding
        self.neuron = LIFNeuron()
        self.input_encoding = encoding.PoissonEncoder()


        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Define fully connected layers
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        # Input encoding
        x = self.input_encoding(x)

        # Convolutional layers
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))

        # Flatten for fully connected layers
        x = x.view(-1, 128 * 8 * 8)

        # LIF Neuron layer
        x = self.neuron(x)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x

# Initialize the SNN
input_size = (3, 32, 32)  # Assuming CIFAR-10 image dimensions
num_classes = 10
snn = SNN(input_size, num_classes)

# Print the SNN architecture
print(snn)

SNN(
  (neuron): LIFNeuron()
  (input_encoding): PoissonEncoder()
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [12]:
from torch.utils.data import DataLoader

# Set batch size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


In [8]:
# Define the loss function
criterion = nn.CrossEntropyLoss()


In [9]:
import torch.optim as optim

# Define the optimizer
optimizer = optim.Adam(snn.parameters(), lr=0.001)  # Adjust the learning rate as needed



In [25]:
# Number of training epochs
num_epochs = 100

for epoch in range(num_epochs):
    snn.train()  # Set the model to training mode
    for (inputs, labels) in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = snn(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    # Validation
    snn.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = snn(inputs)
            val_loss = criterion(outputs, labels)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Val Loss: {val_loss.item()}")


ValueError: too many values to unpack (expected 2)

In [13]:
# Assuming we have loaded your datasets into train_loader and val_loader

# Example for training data
inputs, labels = next(iter(train_loader))
print("Training data shape - Inputs:", inputs.shape, "Labels:", labels.shape)

# Example for validation data
val_inputs, val_labels = next(iter(val_loader))
print("Validation data shape - Inputs:", val_inputs.shape, "Labels:", val_labels.shape)


FileNotFoundError: [Errno 2] No such file or directory: '/content/CIFAR/frames_number_10_split_by_number/automobile/cifar10_automobile_298.npz'

In [ ]:
content/CIFAR/frames_number_10_split_by_number/airplane/cifar10_airplane_0.npz

In [23]:
import torch
from torch.utils.data import DataLoader

# Load train and validation datasets
train_dataset = "train"
val_dataset = "validation"

# Set batch size
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)




In [ ]:
import torch

# Load train and validation datasets
train_dataset = torch.load("final_project/train_dataset.pth")
val_dataset = torch.load("final_project/val_dataset.pth")

# Print the content of the datasets
print("Train Dataset:", train_dataset)
print("Val Dataset:", val_dataset)

# Access the indices of the subsets
train_indices = train_dataset.indices
val_indices = val_dataset.indices

# Print a few example indices
print("Train Indices:", train_indices[:5])
print("Val Indices:", val_indices[:5])



In [15]:
import os
import random
import shutil

# Path to the root directory containing the 10 category folders
root_directory = "frames_number_10_split_by_number"

# Create directories for the training and validation sets
train_directory = "train"
val_directory = "validation"
os.makedirs(train_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

# Percentage of data to be used for validation
validation_percentage = 0.3

# Iterate through each category folder
for category_folder in os.listdir(root_directory):
    category_path = os.path.join(root_directory, category_folder)

    # Get the list of files in the category folder
    files = os.listdir(category_path)

    # Calculate the number of files to be moved to the validation set
    num_val_files = int(len(files) * validation_percentage)

    # Randomly select files for the validation set
    val_files = random.sample(files, num_val_files)

    # Move the selected files to the validation set directory
    for file in val_files:
        src_path = os.path.join(category_path, file)
        dest_path = os.path.join(val_directory, category_folder, file)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.move(src_path, dest_path)

# The remaining files in each category folder are part of the training set
# You can optionally move them to the training set directory
for category_folder in os.listdir(root_directory):
    category_path = os.path.join(root_directory, category_folder)

    # Get the list of files in the category folder
    files = os.listdir(category_path)

    # Move the remaining files to the training set directory
    for file in files:
        src_path = os.path.join(category_path, file)
        dest_path = os.path.join(train_directory, category_folder, file)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.move(src_path, dest_path)


In [20]:
import os

# Replace "path/to/your/data" with the actual path to your data
data_path = "train"

# Loop through each category folder
for category_folder in os.listdir(data_path):
    category_path = os.path.join(data_path, category_folder)

    # List all files in the category folder
    files = os.listdir(category_path)

    # Count the number of files in the category folder
    num_files = len(files)

    print(f"Number of files in {category_folder}: {num_files}")



Number of files in automobile: 700
Number of files in dog: 700
Number of files in ship: 700
Number of files in airplane: 700
Number of files in cat: 700
Number of files in deer: 700
Number of files in truck: 700
Number of files in frog: 700
Number of files in horse: 700
Number of files in bird: 701
